In [ ]:
import tensorflow as tf

2023-09-05 15:43:13.027986: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-05 15:43:13.116918: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-05 15:43:13.118130: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-05 15:43:14.355636: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Define Hyper-Parameters

In [ ]:
PREPROCESSING_ARGS = {
    'sampling_rate': 16000,
    'frame_length_in_s': 0.04,
    'frame_step_in_s': 0.02,
    'num_mel_bins': 40,
    'lower_frequency': 20,
    'upper_frequency': 4000,
}

TRAINING_ARGS = {
    'batch_size': 20,
    'initial_learning_rate': 0.01,
    'end_learning_rate': 1.e-5,
    'epochs': 10
}

final_sparsity = 0.70

# Create Train/Val/Test Datasets

In [ ]:
train_ds = tf.data.Dataset.list_files('msc-train/*')
val_ds = tf.data.Dataset.list_files('msc-val/*')
test_ds = tf.data.Dataset.list_files('msc-test/*')

NameError: name 'tf' is not defined

In [ ]:
from preprocessing import LABELS
from preprocessing import AudioReader
from preprocessing import MelSpectrogram


audio_reader = AudioReader(tf.int16, 16000)
mel_spec_processor = MelSpectrogram(**PREPROCESSING_ARGS)

def prepare_for_training(feature, label):
    feature = tf.expand_dims(feature, -1)
    label_id = tf.argmax(label == LABELS)

    return feature, label_id


batch_size = TRAINING_ARGS['batch_size']
epochs = TRAINING_ARGS['epochs']

train_ds = (train_ds
            .map(audio_reader.get_audio_and_label)
            .map(mel_spec_processor.get_mel_spec_and_label)
            .map(prepare_for_training)
            .batch(batch_size)
            .cache())
val_ds = (val_ds
            .map(audio_reader.get_audio_and_label)
            .map(mel_spec_processor.get_mel_spec_and_label)
            .map(prepare_for_training)
            .batch(batch_size))
test_ds = (test_ds
            .map(audio_reader.get_audio_and_label)
            .map(mel_spec_processor.get_mel_spec_and_label)
            .map(prepare_for_training)
            .batch(batch_size))

2023-09-05 15:43:23.857396: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available
2023-09-05 15:43:23.857560: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: AVX AVX2 AVX512F FMA


In [ ]:
for example_batch, example_labels in train_ds.take(1):
  print('Batch Shape:', example_batch.shape)
  print('Data Shape:', example_batch.shape[1:])
  print('Labels:', example_labels)

Batch Shape: (20, 49, 40, 1)
Data Shape: (49, 40, 1)
Labels: tf.Tensor([0 4 4 1 3 3 7 3 2 1 2 2 3 3 7 2 5 5 5 2], shape=(20,), dtype=int64)
2023-09-05 15:43:36.389536: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


# Create the Model


Conv2D(filters=128, kernel_size=[3, 3], stride=[2, 2], use_bias=False, padding=’valid’)
BatchNormalization()
ReLU()
Conv2D(filters=128, kernel_size=[3, 3], stride=[1, 1], use_bias=False, padding=’same’)
BatchNormalization()
ReLU()
Conv2D(filters=128, kernel_size=[3, 3], stride=[1, 1], use_bias=False, padding=’same’)
BatchNormalization()
ReLU()
GlobalAveragePooling2D()
Dense(units=8)
Softmax()


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=example_batch.shape[1:]),
    tf.keras.layers.Conv2D(filters=128, kernel_size=[3, 3], strides=[2, 2], use_bias=False, padding='valid'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(filters=128, kernel_size=[3, 3], strides=[1, 1], use_bias=False, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(filters=128, kernel_size=[3, 3], strides=[1, 1], use_bias=False, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(units=len(LABELS)),
    tf.keras.layers.Softmax()
])

# Setup Magnitude-based Weights Pruning

In [ ]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

begin_step = int(len(train_ds) * epochs * 0.2)
end_step = int(len(train_ds) * epochs)

pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.20,
        final_sparsity=final_sparsity,
        begin_step=begin_step,
        end_step=end_step
    )
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

In [ ]:
model_for_pruning.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_conv2d  (None, 24, 19, 128)       2306      
  (PruneLowMagnitude)                                            
                                                                 
 prune_low_magnitude_batch_  (None, 24, 19, 128)       513       
 normalization (PruneLowMag                                      
 nitude)                                                         
                                                                 
 prune_low_magnitude_re_lu   (None, 24, 19, 128)       1         
 (PruneLowMagnitude)                                             
                                                                 
 prune_low_magnitude_conv2d  (None, 24, 19, 128)       294914    
 _1 (PruneLowMagnitude)                                          
                                                        

# Train the Model

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=False)
initial_learning_rate = TRAINING_ARGS['initial_learning_rate']
end_learning_rate = TRAINING_ARGS['end_learning_rate']

linear_decay = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=initial_learning_rate,
    end_learning_rate=end_learning_rate,
    decay_steps=len(train_ds) * epochs,
)
optimizer = tf.optimizers.Adam(learning_rate=linear_decay)
metrics = [tf.metrics.SparseCategoricalAccuracy()]
callbacks = [tfmot.sparsity.keras.UpdatePruningStep()]
model_for_pruning.compile(loss=loss, optimizer=optimizer, metrics=metrics)

history = model_for_pruning.fit(train_ds, epochs=epochs, validation_data=val_ds, callbacks=callbacks)

Epoch 1/10
320/320 [==============================] - 246s 754ms/step - loss: 1.9533 - sparse_categorical_accuracy: 0.2300 - val_loss: 1.8354 - val_sparse_categorical_accuracy: 0.2788
Epoch 2/10
320/320 [==============================] - 149s 466ms/step - loss: 1.4109 - sparse_categorical_accuracy: 0.4748 - val_loss: 1.5679 - val_sparse_categorical_accuracy: 0.4725
Epoch 3/10
320/320 [==============================] - 149s 465ms/step - loss: 0.9528 - sparse_categorical_accuracy: 0.6644 - val_loss: 0.9830 - val_sparse_categorical_accuracy: 0.6600
Epoch 4/10
320/320 [==============================] - 151s 471ms/step - loss: 0.7019 - sparse_categorical_accuracy: 0.7659 - val_loss: 0.6670 - val_sparse_categorical_accuracy: 0.8012
Epoch 5/10
320/320 [==============================] - 168s 524ms/step - loss: 0.5569 - sparse_categorical_accuracy: 0.8155 - val_loss: 0.6253 - val_sparse_categorical_accuracy: 0.7962
Epoch 6/10
320/320 [==============================] - 152s 474ms/step - loss: 0.

In [ ]:
history.history

{'loss': [1.953321933746338,
  1.4109406471252441,
  0.9527828097343445,
  0.7018911242485046,
  0.5568985939025879,
  0.476011723279953,
  0.4168280363082886,
  0.36837804317474365,
  0.33201244473457336,
  0.30442625284194946],
 'sparse_categorical_accuracy': [0.23000000417232513,
  0.47484374046325684,
  0.6643750071525574,
  0.7659375071525574,
  0.8154687285423279,
  0.8498437404632568,
  0.8671875,
  0.8884375095367432,
  0.8973437547683716,
  0.9115625023841858],
 'val_loss': [1.835437297821045,
  1.567899465560913,
  0.9830249547958374,
  0.6669675707817078,
  0.6253290176391602,
  0.5737168788909912,
  0.7121668457984924,
  0.5369672179222107,
  0.40990573167800903,
  0.384001761674881],
 'val_sparse_categorical_accuracy': [0.2787500023841858,
  0.4724999964237213,
  0.6600000262260437,
  0.8012499809265137,
  0.7962499856948853,
  0.8137500286102295,
  0.78125,
  0.8274999856948853,
  0.8687499761581421,
  0.8774999976158142]}

In [ ]:
import numpy as np


for layer in model_for_pruning.layers:
    if isinstance(layer, tf.keras.layers.Wrapper):
        weights = layer.trainable_weights
    else:
        weights = layer.weights
    for weight in weights:        
        weight_size = weight.numpy().size
        zero_num = np.count_nonzero(weight == 0)
        print(
            f'{weight.name}: {zero_num/weight_size:.2%} sparsity ',
            f'({zero_num}/{weight_size})',
        )

conv2d/kernel:0: 69.97% sparsity  (806/1152)
batch_normalization/gamma:0: 0.00% sparsity  (0/128)
batch_normalization/beta:0: 0.00% sparsity  (0/128)
conv2d_1/kernel:0: 70.00% sparsity  (103218/147456)
batch_normalization_1/gamma:0: 0.00% sparsity  (0/128)
batch_normalization_1/beta:0: 0.00% sparsity  (0/128)
conv2d_2/kernel:0: 70.00% sparsity  (103218/147456)
batch_normalization_2/gamma:0: 0.00% sparsity  (0/128)
batch_normalization_2/beta:0: 0.00% sparsity  (0/128)
dense/kernel:0: 70.02% sparsity  (717/1024)
dense/bias:0: 0.00% sparsity  (0/8)


# Test the model

In [ ]:
test_loss, test_accuracy = model_for_pruning.evaluate(test_ds)

40/40 [==============================] - 15s 367ms/step - loss: 0.3401 - sparse_categorical_accuracy: 0.8900


In [ ]:
training_loss = history.history['loss'][-1]
training_accuracy = history.history['sparse_categorical_accuracy'][-1]
val_loss = history.history['val_loss'][-1]
val_accuracy = history.history['val_sparse_categorical_accuracy'][-1]

print(f'Training Loss: {training_loss:.4f}')
print(f'Training Accuracy: {training_accuracy*100.:.2f}%')
print()
print(f'Validation Loss: {val_loss:.4f}')
print(f'Validation Accuracy: {val_accuracy*100.:.2f}%')
print()
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy*100.:.2f}%')

Training Loss: 0.3044
Training Accuracy: 91.16%

Validation Loss: 0.3840
Validation Accuracy: 87.75%

Test Loss: 0.3401
Test Accuracy: 89.00%


# Save the Model

### Save Keras Model

In [ ]:
import os
from time import time

timestamp = int(time())

model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)
saved_model_dir = f'./saved_models/{timestamp}'
if not os.path.exists(saved_model_dir):
    os.makedirs(saved_model_dir)
model_for_export.save(saved_model_dir)

INFO:tensorflow:Assets written to: ./saved_models/1693930644/assets
INFO:tensorflow:Assets written to: ./saved_models/1693930644/assets


### Save Hyper-Parameters and Results

In [ ]:
import pandas as pd

output_dict = {
    'timestamp': timestamp,
    **PREPROCESSING_ARGS,
    **TRAINING_ARGS,
    'final_sparsity': final_sparsity,
    'test_accuracy': test_accuracy
}

df = pd.DataFrame([output_dict])

output_path='./spectrogram_wp_results.csv'
df.to_csv(output_path, mode='a', header=not os.path.exists(output_path), index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6f1fd91f-a434-4542-983d-3ce5ae14ac33' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>